# Working with OpenAI in Pixeltable

Pixeltable's OpenAI integration enables you to access OpenAI models via the OpenAI API.

### Prerequisites

- An OpenAI account with an API key (https://openai.com/index/openai-api/)

### Important notes

- OpenAI usage may incur costs based on your OpenAI plan.
- Be mindful of sensitive data and consider security measures when integrating with external services.

First you'll need to install required libraries and enter your OpenAI API key.

In [ ]:
%pip install -qU pixeltable openai

In [ ]:
import os
import getpass
if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter your OpenAI API key:')

Now let's create a Pixeltable directory to hold the tables for our demo.

In [ ]:
import pixeltable as pxt

# Remove the 'openai_demo' directory and its contents, if it exists
pxt.drop_dir('openai_demo', force=True)
pxt.create_dir('openai_demo')

## Chat completions

Create a Table: In Pixeltable, create a table with columns to represent your input data and the columns where you want to store the results from OpenAI.

In [ ]:
from pixeltable.functions import openai

# Create a table in Pixeltable and add a computed column that calls OpenAI

t = pxt.create_table('openai_demo.chat', {'input': pxt.String})

messages = [{'role': 'user', 'content': t.input}]
t.add_computed_column(output=openai.chat_completions(
    messages=messages,
    model='gpt-4o-mini',
    model_kwargs={
        # Optional dict with parameters for the OpenAI API
        'max_tokens': 300,
        'top_p': 0.9,
        'temperature': 0.7
    }
))

In [ ]:
# Parse the response into a new column
t.add_computed_column(response=t.output.choices[0].message.content)

In [ ]:
# Start a conversation
t.insert([{'input': 'How many islands are in the Aleutian island chain?'}])
t.select(t.input, t.response).head()

## Embeddings

Note: OpenAI Embeddings API is not available with free tier API keys

In [ ]:
emb_t = pxt.create_table('openai_demo.embeddings', {'input': pxt.String})
emb_t.add_computed_column(embedding=openai.embeddings(
    input=emb_t.input,
    model='text-embedding-3-small'
))

In [ ]:
emb_t.insert([{'input': 'OpenAI provides a variety of embeddings models.'}])

In [ ]:
emb_t.head()

## Image generations

In [ ]:
image_t = pxt.create_table('openai_demo.images', {'input': pxt.String})
image_t.add_computed_column(img=openai.image_generations(
    image_t.input,
    model='dall-e-2',
))

In [ ]:
image_t.insert([
    {'input': 'A giant Pixel floating in the open ocean in a sea of data'}
])

In [ ]:
image_t

In [ ]:
image_t.head()

## Audio Transcription

In [ ]:
audio_t = pxt.create_table('openai_demo.audio', {'input': pxt.Audio})
audio_t.add_computed_column(result=openai.transcriptions(
    audio_t.input,
    model='whisper-1',
    model_kwargs={
        'language': 'en',
        'prompt': 'Transcribe the contents of this recording.'
    },
))

In [ ]:
url = (
    'https://github.com/pixeltable/pixeltable/raw/release/tests/data/audio/'
    'jfk_1961_0109_cityuponahill-excerpt.flac'
)
audio_t.insert([{'input': url}])

In [ ]:
audio_t.head()

In [ ]:
audio_t.head()[0]['result']['text']

### Learn more

To learn more about advanced techniques like RAG operations in Pixeltable, check out the [RAG Operations in Pixeltable](https://docs.pixeltable.com/howto/use-cases/rag-operations) tutorial.

If you have any questions, don't hesitate to reach out.